In [ ]:
import os
import re
import gc
import glob
import imageio
import numpy as np
import pandas as pd 
from tqdm import tqdm
import pydicom as dicom
from skimage.transform import resize

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from sklearn.model_selection import train_test_split

In [ ]:
FF = 10
BS = 12
IS = 224
LS = 50

In [ ]:
def get_patient_id(raw_ids):
    list=[]
    for patient_id in raw_ids:
        patient_id = int(patient_id)
        if patient_id < 10:
            list.append('0000'+str(patient_id))
        elif patient_id >= 10 and patient_id < 100:
            list.append('000'+str(patient_id))
        elif patient_id >= 100 and patient_id < 1000:
            list.append('00'+str(patient_id))
        else:
            list.append('0'+str(patient_id))
    return np.array(list)


def get_path(row, test):
    nid = row.BraTS21ID
    path = str(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{nid}/{test}')
    return path

In [ ]:
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
df = df.reset_index(drop=True)
df["BraTS21ID"] = get_patient_id(df["BraTS21ID"])

df_FLAIR = df.copy()
df_FLAIR["path"] = df_FLAIR.apply(lambda row: get_path(row, 'FLAIR'), axis=1)
df_T1w = df.copy()
df_T1w["path"] = df_T1w.apply(lambda row: get_path(row, 'T1w'), axis=1)
df_T1wCE = df.copy()
df_T1wCE["path"] = df_T1wCE.apply(lambda row: get_path(row, 'T1wCE'), axis=1)
df_T2w = df.copy()
df_T2w["path"] = df_T2w.apply(lambda row: get_path(row, 'T2w'), axis=1)

In [ ]:
train_FLAIR, valid_FLAIR = train_test_split(df_FLAIR, test_size=0.1, stratify=df_FLAIR.MGMT_value.values, random_state=42)
train_T1w, valid_T1w = train_test_split(df_T1w, test_size=0.1, stratify=df_T1w.MGMT_value.values, random_state=42)
train_T1wCE, valid_T1wCE = train_test_split(df_T1wCE, test_size=0.1, stratify=df_T1wCE.MGMT_value.values, random_state=42)
train_T2w, valid_T2w = train_test_split(df_T2w, test_size=0.1, stratify=df_T2w.MGMT_value.values, random_state=42)
train_label, valid_label = train_test_split(df_FLAIR.MGMT_value.values, test_size=0.1, stratify=df_FLAIR.MGMT_value.values, random_state=42)

In [ ]:
def sorted_nicely(l): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

def decode_image(image):
    image = dicom.dcmread(image)
    image = image.pixel_array
    return image

def parse_frames(dirname):
    paths = glob.glob(dirname.decode('utf8')+'/*.dcm')
    paths = sorted_nicely(paths)
    if len(paths)<FF:
        mri_images = np.zeros(shape=(FF, IS, IS, 3),dtype=np.float32)
    else:
        start = tf.random.uniform((1,), maxval=len(paths)-FF, dtype=tf.int32)
        paths = tf.slice(paths, start, [FF])
    
        def get_frames(path):
            path = str(path)
            pathx_list = path.split("'")
            pathx = pathx_list[1]
            image = decode_image(pathx)
            image = resize(image, (IS, IS, 1), anti_aliasing=True)
            image = image.repeat(3, axis=-1)
            image = tf.convert_to_tensor(image, dtype=tf.float32)
            return image

        mri_images = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn=get_frames, elems=paths, fn_output_signature=tf.float32))
    return mri_images

def load_frame(df_dict):
    dirname = df_dict['path']
    paths = tf.numpy_function(parse_frames, [dirname], tf.float32)   
    label = df_dict['MGMT_value']
    label = tf.cast(label, tf.float32)
    
    return paths, label

In [ ]:
def test_load_frame(df_dict):
    dirname = df_dict['path']
    paths = tf.numpy_function(parse_frames, [dirname], tf.float32)  
    
    return paths

In [ ]:
FLAIR_t_loader = tf.data.Dataset.from_tensor_slices(dict(train_FLAIR))
FLAIR_v_loader = tf.data.Dataset.from_tensor_slices(dict(valid_FLAIR))

T1w_t_loader = tf.data.Dataset.from_tensor_slices(dict(train_T1w))
T1w_v_loader = tf.data.Dataset.from_tensor_slices(dict(valid_T1w))

T1wCE_t_loader = tf.data.Dataset.from_tensor_slices(dict(train_T1wCE))
T1wCE_v_loader = tf.data.Dataset.from_tensor_slices(dict(valid_T1wCE))

T2w_t_loader = tf.data.Dataset.from_tensor_slices(dict(train_T2w))
T2w_v_loader = tf.data.Dataset.from_tensor_slices(dict(valid_T2w))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

FLAIR_t_loader = (
    FLAIR_t_loader
    .shuffle(1024)
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

FLAIR_v_loader = (
    FLAIR_v_loader
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
T1w_t_loader = (
    T1w_t_loader
    .shuffle(1024)
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

T1w_v_loader = (
    T1w_v_loader
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
T1wCE_t_loader = (
    T1wCE_t_loader
    .shuffle(1024)
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

T1wCE_v_loader = (
    T1wCE_v_loader
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
T2w_t_loader = (
    T2w_t_loader
    .shuffle(1024)
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

T2w_v_loader = (
    T2w_v_loader
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
example = next(iter(T2w_t_loader))[0]
example.shape

# **MODEL and TRAINING**

In [ ]:
import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')

import efficientnet.tfkeras as efn

def Eff():  
    base_model = efn.EfficientNetB0(include_top=False, weights=None)
    base_model.load_weights('../input/efficientnetb0b7-keras-weights/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5')
    
    base_model.trainable = True

    inputs = Input((IS, IS, 3))
    x = base_model(inputs, training=True)
    flattened_output = GlobalAveragePooling2D()(x)
    
    return Model(inputs, flattened_output)

In [ ]:
def MainModel():
    input = Input([FF, IS, IS, 3])
    eff = Eff()
    
    time_wrapper = TimeDistributed(eff)(input)
    
    lstm = LSTM(LS, return_sequences=True, name="lstm")(time_wrapper)
    output = Dense(1, activation='sigmoid', name="lstm_sigmoid")(lstm)
    
    return Model(input, output)

In [ ]:
tf.keras.backend.clear_session() 
model_FLAIR = MainModel()
model_T1w = MainModel()
model_T1wCE = MainModel()
model_T2w = MainModel()

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                          patience=5, 
                                          restore_best_weights=True)

save_FLAIR = tf.keras.callbacks.ModelCheckpoint('./FLAIR', 
                                               monitor='val_loss', 
                                               save_best_only=True, 
                                               save_freq='epoch',)
save_T1w = tf.keras.callbacks.ModelCheckpoint('./T1w', 
                                               monitor='val_loss', 
                                               save_best_only=True, 
                                               save_freq='epoch',)
save_T1wCE = tf.keras.callbacks.ModelCheckpoint('./T1wCE', 
                                               monitor='val_loss', 
                                               save_best_only=True, 
                                               save_freq='epoch',)
save_T2w = tf.keras.callbacks.ModelCheckpoint('./T2w', 
                                               monitor='val_loss', 
                                               save_best_only=True, 
                                               save_freq='epoch',)

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, 2, 0.1)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

model_FLAIR.compile(loss="binary_crossentropy", optimizer=optimizer, metrics="acc")
model_T1w.compile(loss="binary_crossentropy", optimizer=optimizer, metrics="acc")
model_T1wCE.compile(loss="binary_crossentropy", optimizer=optimizer, metrics="acc")
model_T2w.compile(loss="binary_crossentropy", optimizer=optimizer, metrics="acc")

In [ ]:
model_T1w.fit(
    T1w_t_loader,
    epochs=100, 
    callbacks=[save_T1w, earlystop], 
    validation_data=(T1w_v_loader), 
              )

In [ ]:
model_FLAIR.fit(
    FLAIR_t_loader,
    epochs=100, 
    callbacks=[save_FLAIR, earlystop], 
    validation_data=(FLAIR_v_loader), 
              )

In [ ]:
model_T1wCE.fit(
    T1wCE_t_loader,
    epochs=100, 
    callbacks=[save_T1wCE, earlystop], 
    validation_data=(T1wCE_v_loader), 
              )

In [ ]:
model_T2w.fit(
    T2w_t_loader,
    epochs=100, 
    callbacks=[save_T2w, earlystop], 
    validation_data=(T2w_v_loader), 
              )

# concate model

# **TEST**

In [ ]:
test_array = [f.name for f in os.scandir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test') if f.is_dir()]
test_array = sorted_nicely(test_array)
test_df = pd.DataFrame({"BraTS21ID":test_array})
test_FLAIR = test_df.copy()
test_FLAIR["path"] = test_df.apply(lambda row: get_path(row, 'FLAIR'), axis=1)
test_T1w = test_df.copy()
test_T1w["path"] = test_T1w.apply(lambda row: get_path(row, 'T1w'), axis=1)
test_T1wCE = test_df.copy()
test_T1wCE["path"] = test_T1wCE.apply(lambda row: get_path(row, 'T1wCE'), axis=1)
test_T2w = test_df.copy()
test_T2w["path"] = test_T2w.apply(lambda row: get_path(row, 'T2w'), axis=1)

In [ ]:
FLAIR_test_loader = tf.data.Dataset.from_tensor_slices(dict(test_FLAIR))
T1w_test_loader = tf.data.Dataset.from_tensor_slices(dict(test_T1w))
T1wCE_test_loader = tf.data.Dataset.from_tensor_slices(dict(test_T1wCE))
T2w_test_loader = tf.data.Dataset.from_tensor_slices(dict(test_T2w))

In [ ]:
FLAIR_test_loader = (
    FLAIR_test_loader
    .map(test_load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)
T1wCE_test_loader = (
    T1wCE_test_loader
    .map(test_load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)
T1w_test_loader = (
    T1w_test_loader
    .map(test_load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)
T2w_test_loader = (
    T2w_test_loader
    .map(test_load_frame, num_parallel_calls=AUTOTUNE)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
FLAIRpred = model_FLAIR.predict(FLAIR_test_loader)
T1wpred = model_T1w.predict(T1w_test_loader)
T1wCEpred = model_T1wCE.predict(T1wCE_test_loader)
T2wpred = model_T2w.predict(T2w_test_loader)

In [ ]:
np.save("./FLAIRpred.npy", FLAIRpred)
np.save("./T1wpred.npy", T1wpred)
np.save("./T1wCEpred.npy", T1wCEpred)
np.save("./T2wpred.npy", T2wpred)

In [ ]:
"""fin_pred = np.concatenate([FLAIRpred,T1wpred,T1wCEpred,T2wpred], axis=-1)
fin_pred = np.mean(fin_pred, axis = -1)
fin_pred = np.mean(fin_pred, axis = 1)"""

In [ ]:
fin_pred = FLAIRpred+T1wpred+T1wCEpred+T2wpred
fin_pred = fin_pred.sum(axis=1)
fmax = fin_pred.max()
fmin = fin_pred.min()
diff = fmax - fmin
fmean = fin_pred.mean()
fin_pred = ((fin_pred-fmean)/diff)+0.5
fin_pred = np.reshape(fin_pred, (87))

In [ ]:
for i in range(len(fin_pred)):
    if fin_pred[i]>1:
        fin_pred[i] = 1
    elif fin_pred[i]<0:
        fin_pred[i] = 0

In [ ]:
submission = pd.DataFrame({"BraTS21ID":test_array, "MGMT_value":fin_pred})
submission

In [ ]:
submission.to_csv("submission.csv", index=False)